In [3]:
import ccxt
import time
import datetime
from zoneinfo import ZoneInfo
import concurrent.futures

# Connect to MEXC API
MEXC_API_KEY = 'mx0vglYmgOGC30vOnC'
MEXC_API_SECRET = 'cbf7b2e829dc4cec9aabd811efa809fd'

mexc = ccxt.mexc3({
    'apiKey': MEXC_API_KEY,
    'secret': MEXC_API_SECRET,
    'enableRateLimit': True,
    'options': {'adjustForTimeDifference': True}
})


AttributeError: module 'ccxt' has no attribute 'mexc3'

In [3]:
# Arbitrage settings
SPREAD_THRESHOLD = 0.1 / 100  # 0.1% for testing
MIN_VOLUME = 50000  # Minimum 24h volume required (in USDT)
CHECK_INTERVAL = 5  # Check every 5 seconds
LOCAL_TIMEZONE = ZoneInfo("Asia/Jakarta")  # Change this to your local timezone

# Estimated trading fee
TRADING_FEE = 0.001  # MEXC approx 0.1% per side

# Cache for futures symbols
futures_markets_cache = []

def get_tradable_tokens(exchange):
    try:
        markets = exchange.load_markets()
        tradable_tokens = []

        global futures_markets_cache
        futures_markets_cache = [symbol for symbol, market in markets.items() if market.get('future')]

        for symbol, market in markets.items():
            if not market.get('spot', False):
                continue
            if '/USDT' in symbol:
                try:
                    market_data = exchange.fetch_ticker(symbol)
                    volume = market_data.get('quoteVolume', 0)
                    if volume and volume >= MIN_VOLUME:
                        tradable_tokens.append(symbol)
                except Exception:
                    continue
        print(f"✅ Found {len(tradable_tokens)} tradable tokens on MEXC")
        return tradable_tokens
    except Exception as e:
        print(f"❌ Error fetching tradable tokens from MEXC: {e}")
        return []

def get_price(exchange, symbol):
    try:
        ticker = exchange.fetch_ticker(symbol)
        return ticker.get("last", None)
    except Exception:
        return None

def fetch_prices_concurrently(exchange, symbols):
    prices = {}
    def fetch(symbol):
        prices[symbol] = get_price(exchange, symbol)
    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(fetch, symbols)
    return prices

def find_futures_symbol(spot_symbol):
    base = spot_symbol.split('/')[0]
    match = [f for f in futures_markets_cache if f.startswith(f"{base}/USDT") or f.startswith(f"{base}-USDT")]
    return match[0] if match else None

def check_intra_exchange_arbitrage(exchange, symbol, spot_prices, futures_prices):
    spot_price = spot_prices.get(symbol)
    futures_symbol = find_futures_symbol(symbol)

    if not futures_symbol:
        return

    futures_price = futures_prices.get(futures_symbol)
    if not spot_price or not futures_price:
        return

    spread = (futures_price - spot_price) / spot_price
    net_profit = spread - (TRADING_FEE * 2)

    if net_profit >= SPREAD_THRESHOLD:
        timestamp = datetime.datetime.now(LOCAL_TIMEZONE).strftime('%Y-%m-%d %H:%M:%S %Z')
        print(f"🔍 Arbitrage Opportunity on MEXC: {symbol} | {timestamp}")
        print(f"    ✅ Spot Price: {spot_price:.5f} USDT")
        print(f"    ✅ Futures Price: {futures_price:.5f} USDT")
        print(f"    ✅ Spread: {spread:.4%}")
        print(f"    ✅ Net Profit After Fees: {net_profit:.4%}")
        print(f"    ⚡ Execute arbitrage on MEXC")
        print("-" * 60)

def process_tokens(exchange, tokens):
    futures_symbols = [find_futures_symbol(t) for t in tokens if find_futures_symbol(t)]

    spot_prices = fetch_prices_concurrently(exchange, tokens)
    futures_prices = fetch_prices_concurrently(exchange, futures_symbols)

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(check_intra_exchange_arbitrage, exchange, token, spot_prices, futures_prices) for token in tokens]
        concurrent.futures.wait(futures)

def main():
    while True:
        try:
            tradable_tokens = get_tradable_tokens(mexc)
            if not tradable_tokens:
                print(f"❌ No tradable tokens found on MEXC. Retrying in 10 seconds...")
                time.sleep(10)
                continue
            print(f"🔄 Checking for arbitrage opportunities on MEXC...")
            process_tokens(mexc, tradable_tokens)
            print("✅ Cycle complete. Waiting for next check...")
            time.sleep(CHECK_INTERVAL)
        except Exception as e:
            print(f"❌ Error in main loop: {e}")
            time.sleep(10)

if __name__ == "__main__":
    main()

❌ Error fetching tradable tokens from MEXC: mexc {"code":700003,"msg":"Timestamp for this request is outside of the recvWindow."}
❌ No tradable tokens found on MEXC. Retrying in 10 seconds...
❌ Error fetching tradable tokens from MEXC: mexc {"code":700003,"msg":"Timestamp for this request is outside of the recvWindow."}
❌ No tradable tokens found on MEXC. Retrying in 10 seconds...


KeyboardInterrupt: 

In [4]:
pip install --upgrade ccxt

   ---------------------------------------- 0.0/5.7 MB ? eta -:--:--
   ----------- ---------------------------- 1.6/5.7 MB 7.6 MB/s eta 0:00:01
   --------------------------- ------------ 3.9/5.7 MB 9.0 MB/s eta 0:00:01
   ---------------------------------- ----- 5.0/5.7 MB 8.2 MB/s eta 0:00:01
   ---------------------------------------- 5.7/5.7 MB 8.3 MB/s eta 0:00:00
  Attempting uninstall: ccxt
    Found existing installation: ccxt 4.4.68
    Uninstalling ccxt-4.4.68:
      Successfully uninstalled ccxt-4.4.68
Note: you may need to restart the kernel to use updated packages.


In [2]:
print(ccxt.exchanges)

['ace', 'alpaca', 'ascendex', 'bequant', 'bigone', 'binance', 'binancecoinm', 'binanceus', 'binanceusdm', 'bingx', 'bit2c', 'bitbank', 'bitbns', 'bitcoincom', 'bitfinex', 'bitfinex1', 'bitflyer', 'bitget', 'bithumb', 'bitmart', 'bitmex', 'bitopro', 'bitpanda', 'bitrue', 'bitso', 'bitstamp', 'bitteam', 'bitvavo', 'bl3p', 'blockchaincom', 'blofin', 'btcalpha', 'btcbox', 'btcmarkets', 'btcturk', 'bybit', 'cex', 'coinbase', 'coinbaseadvanced', 'coinbaseexchange', 'coinbaseinternational', 'coincatch', 'coincheck', 'coinex', 'coinlist', 'coinmate', 'coinmetro', 'coinone', 'coinsph', 'coinspot', 'cryptocom', 'cryptomus', 'defx', 'delta', 'deribit', 'derive', 'digifinex', 'ellipx', 'exmo', 'fmfwio', 'gate', 'gateio', 'gemini', 'hashkey', 'hitbtc', 'hollaex', 'htx', 'huobi', 'huobijp', 'hyperliquid', 'idex', 'independentreserve', 'indodax', 'kraken', 'krakenfutures', 'kucoin', 'kucoinfutures', 'kuna', 'latoken', 'lbank', 'luno', 'mercado', 'mexc', 'myokx', 'ndax', 'novadax', 'oceanex', 'okcoin'

In [1]:
import ccxt
print(ccxt.__version__)

4.4.71


In [ ]:
!pip install --upgrade ccxt

In [2]:
import ccxt
print(ccxt.__version__)
print('mexc3' in ccxt.exchanges)

4.4.71
False


In [3]:
import ccxt
print(ccxt.__file__)

C:\Users\ASUS\anaconda3\Lib\site-packages\ccxt\__init__.py


In [4]:
!pip uninstall -y ccxt

Found existing installation: ccxt 4.4.71
Uninstalling ccxt-4.4.71:
  Successfully uninstalled ccxt-4.4.71


In [5]:
!pip install ccxt --upgrade

  Using cached ccxt-4.4.71-py2.py3-none-any.whl.metadata (131 kB)
Using cached ccxt-4.4.71-py2.py3-none-any.whl (5.7 MB)


In [2]:
import ccxt
print(ccxt.__version__)
print('mexc3' in ccxt.exchanges)

4.4.71
False


In [3]:
import ccxt
import os
print(ccxt.__file__)
print(os.listdir(os.path.dirname(ccxt.__file__)))

C:\Users\ASUS\anaconda3\Lib\site-packages\ccxt\__init__.py
['abstract', 'ace.py', 'alpaca.py', 'ascendex.py', 'async_support', 'base', 'bequant.py', 'bigone.py', 'binance.py', 'binancecoinm.py', 'binanceus.py', 'binanceusdm.py', 'bingx.py', 'bit2c.py', 'bitbank.py', 'bitbns.py', 'bitcoincom.py', 'bitfinex.py', 'bitfinex1.py', 'bitflyer.py', 'bitget.py', 'bithumb.py', 'bitmart.py', 'bitmex.py', 'bitopro.py', 'bitpanda.py', 'bitrue.py', 'bitso.py', 'bitstamp.py', 'bitteam.py', 'bitvavo.py', 'bl3p.py', 'blockchaincom.py', 'blofin.py', 'btcalpha.py', 'btcbox.py', 'btcmarkets.py', 'btcturk.py', 'bybit.py', 'cex.py', 'coinbase.py', 'coinbaseadvanced.py', 'coinbaseexchange.py', 'coinbaseinternational.py', 'coincatch.py', 'coincheck.py', 'coinex.py', 'coinlist.py', 'coinmate.py', 'coinmetro.py', 'coinone.py', 'coinsph.py', 'coinspot.py', 'cryptocom.py', 'cryptomus.py', 'defx.py', 'delta.py', 'deribit.py', 'derive.py', 'digifinex.py', 'ellipx.py', 'exmo.py', 'fmfwio.py', 'gate.py', 'gateio.py',